# Part 1.4 - REs and finite state automata

In Natural Language Processing (`NLP`), Finite Automata plays a crucial role in recognizing and processing patterns within text. They are used to model the sequence of states and transitions that a system undergoes in response to a string of inputs, making them particularly effective for tasks like lexical analysis, tokenization, and syntax parsing. The theory of Finite Automata is deeply intertwined with Regular Expressions (`Regex`), as every regular expression can be converted into an equivalent finite automaton, and vice versa. This relationship is significant because it provides a formal and efficient way to implement regex patterns using finite state machines. By understanding Finite Automata, Parham can gain insight into how `Regex` operate at a fundamental level, allowing him to better grasp how patterns are matched and how text is processed in various NLP tasks.


**Objectives:**

By the end of this notebook, Parham will have learned the theory of finite automata and how to apply it in code. He will leverage the well-known Python library, `automata-lib`, to gain hands-on experience with the `Automaton` class. Subsequently, Parham will use the available functions of the library to convert finite automata patterns into `Regex` patterns. Additionally, various exception classes will be discussed to ensure that errors are handled appropriately.

**Tabel of Content:**

1. Import Libraries
2. Introduction to the theory of finit-autonama
3. Class Automation
4. Regular Expressions
5. Exceptions and Errors
6. Closing Thoughts

## 1) Import Libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import automata
from automata.base.automaton import Automaton
from automata.fa.dfa import DFA
from automata.fa.fa import FA
from automata.fa.nfa import NFA
from automata.fa.gnfa import GNFA
from automata.pda.dpda import DPDA
from automata.pda.npda import NPDA
from automata.tm.ntm import NTM
from automata.tm.mntm import MNTM
import automata.regex as re
import automata.base.exceptions as exceptions
import automata.tm.exceptions as tm_exceptions



## 2) Introduction to the Theory of Finite Automata

Finite Automata are a foundational concept in computer science, particularly within formal language theory and natural language processing (NLP). A Finite Automaton is a simple computational model designed to recognize patterns within input data, making it crucial for tasks such as lexical analysis, pattern matching, and syntax parsing. Essentially, a Finite Automaton is an abstract machine that processes a sequence of input symbols, transitioning through a series of states according to a predefined set of rules. The machine comprises the following key components:

- **A finite set of states**: These represent the various configurations in which the machine can exist.
- **An input alphabet**: A finite set of symbols that the machine can read and process.
- **A transition function**: This defines how the machine moves from one state to another based on the current input symbol.
- **A start state**: The initial state where the machine begins processing the input.
- **Accepting (final) states**: Specific states that determine if the input is accepted (i.e., if the pattern is recognized) when the machine finishes processing.

There are two primary types of Finite Automata:

- **Deterministic Finite Automaton (DFA)**: In a DFA, each state has exactly one transition for each possible input symbol. This ensures that for any given state and input, the next state is uniquely determined.

- **Non-deterministic Finite Automaton (NFA)**: In an NFA, a state may have multiple transitions for the same input symbol or may transition to a new state without consuming any input (ε-transitions). This non-determinism allows the NFA to explore multiple computational paths simultaneously.

### Example: Converting a Non-deterministic Finite Automaton (NFA) to a Deterministic Finite Automaton (DFA)

Consider a system designed to detect a specific sequence of heads (H) and tails (T) from a series of coin tosses. The system is particularly interested in identifying a sequence where tails occur in the last two flips:

`sequence = (H , T)* TT`

The goal is to construct a state diagram that recognizes this sequence. The state diagram can be illustrated as follows:

<p align="center"><img src="../imgs/finite_auto_diagram.png" alt="Picture" width="50%" height="10%" style="display: block; margin: 20px auto;"/></p>

In the diagram above, each symbol is defined as follows:
- **a** : "Ends with heads"
- **b** : "Ends with 1 tail"
- **c** : "Ends with 2 tails"
- **e** : "Error"
- **H** : Head
- **T** : Tail

Below is a simplified procedure that outlines how the system processes a sequence of heads (`H`) and tails (`T`) to detect when two consecutive tails (`TT`) occur:

- Start at State `a`:
  - Initial state where the system begins.

- If Input Symbol is `T`:
  - If in State `a`:
    - Transition to State `b`.
  - If in State `b`:
    - Transition to State `c` (final state).

- If Input Symbol is `H`:
  - From Any State (`a`, `b`, or `c`):
    - Transition back to State `a`.
- If Input Symbol is not one of `H` or `T`:
  - From Any State (`a`, `b`, or `c`):
      - Transition back to State `e`.
- If in State `c` (Final State):
  - Sequence "TT" has been detected, meaning the system has recognized the pattern. 

The transition table for such system can be reported as below:
| Current State/ Inputs | H               | T                                          | O.W |
|----------------|-----------------| ----------------------------------------------| -----|
| a  |            a                 |                 b                       | e|
| b    |               a             |               c                     |  e|
| c    |             a               |                 c                   |  e|

In the cell below, the practitioner will implement the above system.

In [1]:
# Define states and transitions for the finite automaton
states = ["Ends with heads", "Ends with 1 tails", "Ends with 2 tails", "Error"]
initial_state = states[0]
accepting_states = [states[2]]  # List of accepting states
input_alphabet = "HT"

# Transition table maps state indices to their respective transitions
transition_table = {
    0: [states[0], states[1]],  # From "Ends with heads", "H" -> state 0, "T" -> state 1
    1: [states[0], states[2]],  # From "Ends with 1 tails", "H" -> state 0, "T" -> state 2
    2: [states[0], states[2]],  # From "Ends with 2 tails", "H" -> state 0, "T" -> state 2
    3: [states[3], states[3]]   # From "Error", always stays in "Error"
}
error_state = states[3]
def return_next_state(current_state, current_input):
    """
    Given the current state and input, return the next state based on the transition table.
    """
    state_index = states.index(current_state)
    input_index = input_alphabet.index(current_input)
    return transition_table[state_index][input_index]

# Initialize the automaton's current state and the input sequence to process
current_state = initial_state
input_sequence = "HTTHTTTHTH"

# Dictionary to track the number of occurrences for each accepting state
accepting_state_info = {state: {'state_name': state, 'count': 0} for state in accepting_states}

# Process the input sequence
for current_input in input_sequence:
    sys.stdout.write(f"Current State: {current_state} | Input: {current_input}")
    if not(current_input in input_alphabet):
        current_state = error_state
    else:
      current_state = return_next_state(current_state, current_input)
    print(f"  ---> Next State: {current_state}")
    
    # Increment the count if the current state is an accepting state
    if current_state in accepting_states:
        accepting_state_info[current_state]['count'] += 1

# Output the results
print("\n********************\nSummary of Accepting States:\n********************")
for state, info in accepting_state_info.items():
    print(f"Accepting State: {info['state_name']} | Number of Occurrences: {info['count']}")


Current State: Ends with heads | Input: H  ---> Next State: Ends with heads
Current State: Ends with heads | Input: T  ---> Next State: Ends with 1 tails
Current State: Ends with 1 tails | Input: T  ---> Next State: Ends with 2 tails
Current State: Ends with 2 tails | Input: H  ---> Next State: Ends with heads
Current State: Ends with heads | Input: T  ---> Next State: Ends with 1 tails
Current State: Ends with 1 tails | Input: T  ---> Next State: Ends with 2 tails
Current State: Ends with 2 tails | Input: T  ---> Next State: Ends with 2 tails
Current State: Ends with 2 tails | Input: H  ---> Next State: Ends with heads
Current State: Ends with heads | Input: T  ---> Next State: Ends with 1 tails
Current State: Ends with 1 tails | Input: H  ---> Next State: Ends with heads

********************
Summary of Accepting States:
********************
Accepting State: Ends with 2 tails | Number of Occurrences: 3


## 3) Class Automation

## 4) Regular Expressions

## 5) Exceptions and Errors

## 6) Closing Thoughts